In [1]:
# Regarding the GO (only stage5):

# 13) only for stage 5 and for the 12 conditions (groseq+ filtered) list with 
# log2FC1 and padj<0.05. 
# So only the genes that overlap with the groseq+ list.(very important). 
#As a background I do not know if it a good idea to use all the groseq+ genes? 
#Let me know what you think

# Regarding promoter and motif analysis:

# the same classes of points 13 and 14

# - promoter analysis +-300bp
# - motif analysis +-300 bp
# - motif analysis +-2kb

# As a background, let's use all the drosophila genes MINUS everything that is up or down regulated in the 24 conditions but with log2FC<-1/>+1 and in that overlaps with groseq+ (very important! just the groseq+ positive!)


In [2]:
# CPE databases (you name it 'promoter_dbs') are only CPE.Haberle_and_Stark/Haberle_and_Stark_NRMCB_2018_Table1_motifs_PFMs.memeand Ramalingam2021/promoter_elements.meme
# TF databases are  JASPAR2020_CORE_insects_non-redundant_pfms_meme.txt , and all the ones from the second snippet
# Zelda MA1462.1 was added specifically for Dafne's study. It makes sense to use with it with the CPE of you are looking at the zld-dependent genes. Otherwise, treat it like a TF database (yet, it's already included in the JASPAR database).
# I have no clue what these all_motifs.memefile is. Please ignore it.

In [3]:
import os
import pandas as pd
import numpy as np
import pybedtools as pbt
import subprocess as sp

In [4]:
root = os.path.join("/data/iovino/group/rabbani/FC_exp9_4_and_FC_exp9_10/")
flanks = {"300":300, "1000":1000, "2000":2000}

In [5]:
# list of all TSSs

In [6]:
# all genes TSS
genes = pd.read_csv(os.path.join( root,"analysis_ensembl96/featureCounts/",
                    "20200708_01_FC_RNA_T180xN174_Ctrl-KD_emb_st2_rep1.counts.txt"), 
                    sep = "\t", skiprows=1,
                    usecols =['Geneid', 'Chr', 'Start', 'End', 'Strand'])
genes['Chr'] = genes['Chr'].str.split(";")
genes['Chr'] = genes['Chr'].str[0]
genes['Start'] = genes['Start'].str.split(";")
genes['Start'] = genes['Start'].str[0]
genes['End'] = genes['End'].str.split(";")
genes['End'] = genes['End'].str[0]
genes['Strand'] = genes['Strand'].str.split(";")
genes['Strand'] = genes['Strand'].str[0]
genes = genes.loc[genes["Chr"].isin(['2R','2L', '3R', '3L', 'X', '4'])]

In [7]:
# read chrom length file
chrom_length = pd.read_csv("/data/repository/organisms/dm6_ensembl/genome_fasta/genome.chrom.sizes",
                           header = None, sep ="\t")
chrom_length = chrom_length.loc[chrom_length[0].isin(['2R','2L', '3R', '3L', 'X', '4'])].T
chrom_length.columns = chrom_length.iloc[0]
chrom_length.drop([0], axis = 0, inplace = True)

In [8]:
# exteded TSS using the flanking bases
df = pd.DataFrame()
extended_tss = {'tss_' + str(i):df for i in flanks.keys()}
for key, flank in flanks.items():
    tss_pos = genes.loc[genes["Strand"] == "+"]
    tss_pos["End"] = tss_pos["Start"].astype('int32')+flank
    tss_pos["Start"] = tss_pos["Start"].astype('int32')-flank
    tss_pos.loc[tss_pos["Start"] < 0, "Start" ] = 0

    tss_neg = genes.loc[genes["Strand"] == "-"]
    tss_neg["Start"] = tss_neg["End"].astype('int32')-flank
    tss_neg.loc[tss_neg["Start"] < 0, "Start"] = 0
    tss_neg["End"] = tss_neg["End"].astype('int32')+flank
    extended_tss["tss_"+key] = pd.concat([tss_pos, tss_neg])
    extended_tss["tss_"+key].sort_values(['Chr', 'Start'], inplace= True)
    # # check for chr length 
    for chrom in np.unique(extended_tss["tss_"+key]["Chr"].values):
        length = chrom_length[chrom].astype('int32').values[0]
        extended_tss["tss_"+key].loc[(extended_tss["tss_"+key]["Chr"] == chrom) & 
                                      (extended_tss["tss_"+key]["End"] > length), "End"] = length - 1

/data/processing3/leily/miniconda3/envs/matplotlib/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/data/processing3/leily/miniconda3/envs/matplotlib/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/data/processing3/leily/miniconda3/envs/matplotlib/lib/python3.6/site-packages/pandas/core/indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.

In [22]:
# get the groseq overlapped files sequences
tsvs_path = os.path.join(root, "downstream_analysis_ensembl96/groseq_overlap/")
for file in os.listdir(tsvs_path):
    if "padj0.05_lfc-1_groseq" in file:
        name = file.split(".tsv")[0]
        df = pd.read_csv(os.path.join(tsvs_path, file),sep = "\t", usecols = ["gene_id"])
        df.rename(columns={"gene_id": "Geneid"}, inplace = True)
        for k, tss in extended_tss.items():
            df_tss = df.merge(tss, on ="Geneid", how="inner") # each kd is merged to its gene coord.
            df_fasta = pbt.BedTool.from_dataframe(df_tss[["Chr","Start","End"]])
            df_fasta = df_fasta.sequence(fi="/data/repository/organisms/dm6_ensembl/"
                                         "genome_fasta/genome.fa",
                                         fo = name+"_"+k+"base_regions.fasta")    

In [10]:
# motif dbs:

In [11]:
# core promotor motifs dbs: (only for +-300)
path_to_repo = "/data/iovino/repository/rauer/Databases/"
path_to_group = "/data/iovino/group/rauer/Domino-H2Av/output3/TF_motif_enrichment/"\
                "motif_database/"
promoter_dbs = [
    path_to_repo+"CPE.Haberle_and_Stark/Haberle_and_Stark_NRMCB_2018_Table1_motifs_PFMs.meme",
    path_to_repo+"Ramalingam2021/promoter_elements.meme",
    path_to_group+"Custom_databases/Zelda_motif/MA1462.1.meme"
    ]

In [12]:
# Fly motif dbs: (only for +-2000)
# path_to_dbs = "/data/iovino/group/rauer/Domino-H2Av/output2/TFmotifs.error/data/"\
#               "MEME_database/motif_databases/FLY/"
# path_to_dbs+"OnTheFly_2014_Drosophila.meme",
# path_to_dbs+"idmmpmm2009.meme",
# path_to_dbs+"flyreg.v2.meme",
# path_to_dbs+"fly_factor_survey.meme",
# path_to_dbs+"dmmpmm2009.meme",
path_to_jasper = "/data/iovino/group/rauer/Domino-H2Av/output3/TF_motif_enrichment/"\
                "motif_database/"
dbs = [
    path_to_jasper+"JASPAR2020/JASPAR2020_CORE_insects_non-redundant_pfms_meme.txt"
    ]

In [13]:
# option 1: Use all groseq+ as bg

In [14]:
# bg_df = pd.read_csv(os.path.join(root,"downstream_analysis/motif_enrichment/csv_lists/",
#                                  "FC1_groseq+.csv"), sep = ";", usecols = ["groseq+"])
# bg_df.columns = ['Geneid']
# for k, tss in extended_tss.items():
#     print(k)
#     df_tss = bg_df.merge(tss, on ="Geneid", how="inner")
#     df_fasta = pbt.BedTool.from_dataframe(df_tss[["Chr","Start","End"]])
#     df_fasta = df_fasta.sequence(fi="/data/repository/organisms/dm6_ensembl/"
#                                     "genome_fasta/genome.fa",
#                                  fo = "groseq_"+k+"base_regions.fasta")  

In [16]:
# for file in os.listdir(os.getcwd()):
#     name = file.split("_tss")[0]
#     if ("padj0.05_lfc-1" in file ) and (file.endswith("_300base_regions.fasta")):
#         cmd = "module load meme;"
#         cmd += "ame --control groseq_tss_300base_regions.fasta "
#         cmd += "-oc cp_motifs/" + name + "_allgroseq+bg_cp_0.3kb "
#         cmd += os.path.join(os.getcwd(), file) + " "
#         cmd += (" ").join(promoter_dbs)
#         print(cmd)
#         sp.check_output(cmd, shell = True)
#     elif ("padj0.05_lfc-1" in file ) and (file.endswith("_2000base_regions.fasta")):
#         cmd = "module load meme;"
#         cmd += "ame --control groseq_tss_2000base_regions.fasta "
#         cmd += "-oc tf_motifs/" + name + "_allgroseq+bg_tf_2kb  "
#         cmd += os.path.join(os.getcwd(), file) + " "
#         cmd += (" ").join(dbs)
#         print(cmd)
#         sp.check_output(cmd, shell = True)      

In [17]:
# option 2: Exclude the foreground from groseq+

In [18]:
# tsvs_path = os.path.join(root, "downstream_analysis_ensembl96/groseq_overlap/")
# bg_df = pd.read_csv(os.path.join(root,"downstream_analysis/motif_enrichment/csv_lists/",
#                                  "FC1_groseq+.csv"), sep = ";", usecols = ["groseq+"])
# bg_df.columns = ['Geneid']
# def get_extended_tss(extended_tss, df, kd):
#     for k, tss in extended_tss.items():
#         df_tss = df.merge(tss, on ="Geneid", how="inner")
#         df_fasta = pbt.BedTool.from_dataframe(df_tss[["Chr","Start","End"]])
#         df_fasta = df_fasta.sequence(fi="/data/repository/organisms/dm6_ensembl/"
#                                         "genome_fasta/genome.fa",
#                                      fo = "groseq_exclude_"+kd+k+"base_regions.fasta")  
# for file in os.listdir(os.getcwd()):
#     name = file.split("_tss")[0]
#     if ("padj0.05_lfc-1" in file ) and (file.endswith("_300base_regions.fasta")):
#         kd_df = pd.read_csv(tsvs_path+name+".tsv", sep = "\t")
#         kd_df.rename(columns={"gene_id":"Geneid"}, inplace =True)
#         this_bg = bg_df.merge(kd_df, how="outer",indicator=True).loc[lambda x : x['_merge']=='left_only']
#         this_bg = this_bg.iloc[:,0:1]
#         get_extended_tss(extended_tss, this_bg, name.split("padj")[0])
#         cmd = "module load meme;"
#         cmd += "ame --control groseq_exclude_"+name.split("padj")[0]+"tss_300base_regions.fasta "
#         cmd += "-oc cp_motifs/" + name + "_groseq+_exclude_"+name.split("padj")[0]+"bg_cp_0.3kb "
#         cmd += os.path.join(os.getcwd(), file) + " "
#         cmd += (" ").join(promoter_dbs)
#         print(cmd)
#         sp.check_output(cmd, shell = True)
#     elif ("padj0.05_lfc-1" in file ) and (file.endswith("_2000base_regions.fasta")):
#         kd_df = pd.read_csv(tsvs_path+name+".tsv", sep = "\t")
#         kd_df.rename(columns={"gene_id":"Geneid"}, inplace =True)
#         this_bg = bg_df.merge(kd_df, how="outer",indicator=True).loc[lambda x : x['_merge']=='left_only']
#         this_bg = this_bg.iloc[:,0:1]
#         get_extended_tss(extended_tss, this_bg, name.split("padj")[0])
#         cmd = "module load meme;"
#         cmd += "ame --control groseq_exclude_"+name.split("padj")[0]+"tss_2000base_regions.fasta "
#         cmd += "-oc tf_motifs/" + name + "_groseq+_exclude_"+name.split("padj")[0]+"bg_tf_2kb  "
#         cmd += os.path.join(os.getcwd(), file) + " "
#         cmd += (" ").join(dbs)
#         print(cmd)
#         sp.check_output(cmd, shell = True) 

In [19]:
# option3: inactive genes as bg

In [20]:
regions_df = pd.read_csv(os.path.join(root, "downstream_analysis_ensembl96",
                                      "Domino-study.grouping_figure3.gene_id.tsv"),
                        sep = "\t")
inactive = regions_df.loc[regions_df["fig3.group"] == "inactive"]
inactive.rename(columns={"gene_id": "Geneid"}, inplace = True)
for k, tss in extended_tss.items():
    df_tss = inactive.merge(tss, on ="Geneid", how="inner")
    df_fasta = pbt.BedTool.from_dataframe(df_tss[["Chr","Start","End"]])
    df_fasta = df_fasta.sequence(fi="/data/repository/organisms/dm6_ensembl/"
                                    "genome_fasta/genome.fa",
                                 fo = "inactvie_"+k+"base_regions.fasta")  

/data/processing3/leily/miniconda3/envs/matplotlib/lib/python3.6/site-packages/pandas/core/frame.py:4301: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [23]:
for file in os.listdir(os.getcwd()):
    name = file.split("_tss")[0]
    if ("padj0.05_lfc-1" in file ) and (file.endswith("_300base_regions.fasta")):
        cmd = "module load meme;"
        cmd += "ame --control inactvie_tss_300base_regions.fasta "
        cmd += "-oc cp_motifs/" + name + "_inactvie+bg_cp_0.3kb "
        cmd += os.path.join(os.getcwd(), file) + " "
        cmd += (" ").join(promoter_dbs)
        print(cmd)
        sp.check_output(cmd, shell = True)
    elif ("padj0.05_lfc-1" in file ) and (file.endswith("_1000base_regions.fasta")):
        cmd = "module load meme;"
        cmd += "ame --control inactvie_tss_1000base_regions.fasta "
        cmd += "-oc tf_motifs/" + name + "_inactive+bg_tf_1kb  "
        cmd += os.path.join(os.getcwd(), file) + " "
        cmd += (" ").join(dbs)
        print(cmd)
        sp.check_output(cmd, shell = True)  

module load meme;ame --control inactvie_tss_300base_regions.fasta -oc cp_motifs/hdac3_down_padj0.05_lfc-1_groseq_inactvie+bg_cp_0.3kb /data/iovino/group/rabbani/FC_exp9_4_and_FC_exp9_10/downstream_analysis_ensembl96/motif_enrichment/hdac3_down_padj0.05_lfc-1_groseq_tss_300base_regions.fasta /data/iovino/repository/rauer/Databases/CPE.Haberle_and_Stark/Haberle_and_Stark_NRMCB_2018_Table1_motifs_PFMs.meme /data/iovino/repository/rauer/Databases/Ramalingam2021/promoter_elements.meme /data/iovino/group/rauer/Domino-H2Av/output3/TF_motif_enrichment/motif_database/Custom_databases/Zelda_motif/MA1462.1.meme
module load meme;ame --control inactvie_tss_300base_regions.fasta -oc cp_motifs/gcn5_down_padj0.05_lfc-1_groseq_inactvie+bg_cp_0.3kb /data/iovino/group/rabbani/FC_exp9_4_and_FC_exp9_10/downstream_analysis_ensembl96/motif_enrichment/gcn5_down_padj0.05_lfc-1_groseq_tss_300base_regions.fasta /data/iovino/repository/rauer/Databases/CPE.Haberle_and_Stark/Haberle_and_Stark_NRMCB_2018_Table1_moti

module load meme;ame --control inactvie_tss_300base_regions.fasta -oc cp_motifs/hdac1_down_padj0.05_lfc-1_groseq_inactvie+bg_cp_0.3kb /data/iovino/group/rabbani/FC_exp9_4_and_FC_exp9_10/downstream_analysis_ensembl96/motif_enrichment/hdac1_down_padj0.05_lfc-1_groseq_tss_300base_regions.fasta /data/iovino/repository/rauer/Databases/CPE.Haberle_and_Stark/Haberle_and_Stark_NRMCB_2018_Table1_motifs_PFMs.meme /data/iovino/repository/rauer/Databases/Ramalingam2021/promoter_elements.meme /data/iovino/group/rauer/Domino-H2Av/output3/TF_motif_enrichment/motif_database/Custom_databases/Zelda_motif/MA1462.1.meme
module load meme;ame --control inactvie_tss_1000base_regions.fasta -oc tf_motifs/hdac1_down_padj0.05_lfc-1_groseq_inactive+bg_tf_1kb  /data/iovino/group/rabbani/FC_exp9_4_and_FC_exp9_10/downstream_analysis_ensembl96/motif_enrichment/hdac1_down_padj0.05_lfc-1_groseq_tss_1000base_regions.fasta /data/iovino/group/rauer/Domino-H2Av/output3/TF_motif_enrichment/motif_database/JASPAR2020/JASPAR20